# Step 1: Read & Process Transcript Files
We first read the transcript text files, grouping together the files in the "Free" group as one set, and the "D-C" group as another set.

To avoid processing errors, we first remove all non-UTF characters in the text (such as distinct opening and closing quotes)
and replace them with corresponding UTF characters. These transcripts also come with the transcriber's observations, such as [laughter], or [silent]. We remove all such observations so that we are only looking at team members' speech.

Finally we remove all stop words

In [1]:
import re
# Locations of input files. These files have been anonymized, and the files themselves will not be available
# in this folder, to preserve the privacy of the teams.
textPathFree = "../../../private/brainstorming_viz_data/free_group/"
textPathDC = "../../../private/brainstorming_viz_data/d-c_group/"

# Teamwise transcript (of divergent processes) of the "Free Brainstorming" group
fileNamesFreeDivergent = ['teamA_part1_divergent.txt', 'teamB_part1_divergent.txt', 'teamC_part1_divergent.txt',
                          'teamD_part1_divergent.txt', 'teamE_part1_divergent.txt', 'teamF_part1_divergent.txt',
                          'teamG_part1_divergent.txt', 'teamH_part1_divergent.txt', 'teamI_part1_divergent.txt']

# Teamwise transcript (of divergent processes) of the "Divergent-Convergent (D-C) Brainstorming" group
fileNamesDCDivergent = ['teamJ_part1_divergent.txt', 'teamJ_part3_divergent.txt', 'teamK_part1_divergent.txt',
                        'teamK_part3_divergent.txt', 'teamL_part1_divergent.txt', 'teamL_part3_divergent.txt', 
                        'teamM_part1_divergent.txt', 'teamM_part2_3_convergent_divergent.txt', 'teamN_part1_divergent.txt',
                        'teamN_part3_divergent.txt', 'teamO_part1_divergent.txt', 'teamO_part2_3_conv_div.txt',
                        'teamP_part1_divergent.txt', 'teamP_part3_divergent.txt', 'teamQ_part1_divergent.txt',
                        'teamQ_part3_4_div_conv.txt', 'teamR_part1_divergent.txt', 'teamR_part3_divergent.txt'] 

# Teamwise transcript (of divergent processes) of the "Free Brainstorming" group
fileNamesFreeConvergent = ['teamA_part2_convergent.txt', 'teamB_part2_convergent.txt', 'teamC_part2_convergent.txt',
                           'teamD_part2_convergent.txt', 'teamE_part2_convergent.txt', 'teamF_part2_convergent.txt',
                           'teamG_part2_convergent.txt', 'teamH_part2_convergent.txt', 'teamI_part2_convergent.txt']

# Teamwise transcript (of divergent processes) of the "Divergent-Convergent (D-C) Brainstorming" group
fileNamesDCConvergent = ['teamJ_part2_convergent.txt', 'teamJ_part4_convergent.txt', 'teamK_part2_convergent.txt',
                         'teamK_part4_convergent.txt', 'teamL_part2_convergent.txt', 'teamL_part4_convergent.txt',
                         'teamM_part4_convergent.txt', 'teamN_part2_convergent.txt', 'teamN_part4_convergent.txt',
                         'teamO_part4_convergent.txt', 'teamP_part2_convergent.txt', 'teamP_part4_convergent.txt',
                         'teamQ_part2_convergent.txt', 'teamR_part2_convergent.txt', 'teamR_part4_convergent.txt']  

def openFiles(fileNamesList, filePath):
    # Given a list of file names and a path for the files, returns a
    # list of file objects
    fObjList = []
    for fileName in fileNamesList:
        fObj = open(filePath + fileName, 'r', encoding='utf8')
        fObjList.append(fObj)
    return fObjList

# Some of these files have non-UTF characters that may need to be removed.
def removeSpecials(filesList):
    # Remove some incompatible unicode characters such as distinct
    # opening and closing quotes, ellipsis etc.
    # Also finally converts all text to lowercase
    outString = ''
    for inFile in filesList:
        # print("reading file",inFile.name.split("/")[-1], "...", end="")
        temp = inFile.read()
        subsFileQuotes = re.sub(u"(\u2018|\u2019)", "'", temp)
        subsFileDQuotes = re.sub(u'(\u201c|\u201d)', '"', subsFileQuotes)
        subsFileEllipsis = re.sub(u"\u2026", "...", subsFileDQuotes)
        outString += subsFileEllipsis
        # print("done.")
    return outString.lower()

# These files have the transcriber's observations like [laughter], [silent], [cross-talk] etc.
# These may throw off the text analysis, so we remove them. These are typically within parantheses,
# so we can remove everything between these parantheses.

def removeObservations(test_str) :
    # removes all text within parantheses
    # these are usually observations by the transcriber
    # and thus should not be considered in content analysis
    # code from http://stackoverflow.com/questions/14596884/remove-text-between-and-in-python
    ret = ''
    skip1c = 0
    skip2c = 0
    squareparen = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
            squareparen += 1
        elif i == '(':
            skip2c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif i == ')'and skip2c > 0:
            skip2c -= 1
        elif skip1c == 0 and skip2c == 0:
            ret += i
    return ret

filesListFree = openFiles(fileNamesFreeDivergent + fileNamesFreeConvergent, textPathFree)
filesListDC = openFiles(fileNamesDCDivergent + fileNamesDCConvergent, textPathDC)

group1_raw = removeSpecials(filesListFree)
group2_raw = removeSpecials(filesListDC)


group1 = removeObservations(group1_raw)
group2 = removeObservations(group2_raw)

group1_lines = group1.split('\n')
group2_lines = group2.split('\n')

# Step 2: Computing the Latent Dirichlet Allocation
Detailed explanation to follow...


In [2]:
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords

C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use 

## Step 2.1 Computing the Document-Term Matrix

The document-term matrix looks like this:

|      | T1     | T2     | T3     | T4     | T5     |... 
|:----:|:------:|:------:|:------:|:------:|:------:|:---:
| D1   | N_d1t1 | N_d1t2 | N_d1t2 | N_d1t3 | N_d1t4 |...
| D2   | N_d2t1 | N_d2t2 | N_d2t2 | N_d2t3 | N_d2t4 |...
| D3   | N_d3t1 | N_d3t2 | N_d3t2 | N_d3t3 | N_d3t4 |...
| D4   | N_d4t1 | N_d4t2 | N_d4t2 | N_d4t3 | N_d4t4 |...
| D5   | N_d5t1 | N_d5t2 | N_d5t2 | N_d5t3 | N_d5t4 |...
| D6   | N_d6t1 | N_d6t2 | N_d6t2 | N_d6t3 | N_d6t4 |...
| ..   | ...... | ...... | ...... | ...... | ...... |...

Where N_ditj is called the **term frequency**, and is equal to the the number of occurrences of term Tj in document Di.

For our purpose, each term is a word longer than two letters, and each document is a line in the transcript. We need to focus on words that matter, so this means removing any stop words, as well as words from the transcript that we deem irrelevant. These include filler words such as "yeah", "like", etc. and non-words such as "hmm". Since this is a transcript, we find a number of time-stamped entries embedded in the text, so these need to go as well. A complete list of words to remove from the list of terms is below (excluding stop words).

Each document is represented as an M-dimensional vector where M is the number of terms computed for the corpus. Here, the vector corresponds to the row of the document-term matrix.

In [3]:
words_to_remove = ['yea', 'like', 'up', 'down', "!", "$", "'", "''", "'cause", 
                   "'d", "'em", "'ll", "'m", "'re", "'s", "'ve", ",", "yeah",
                   "-", ".", "1", "10", "12", "12.", "15", "20", "3-d", 
                   "30", "300", "352.", "36", "375", "3d", "452", "475", 
                   "475.", "90", ":", ";", "?", "``", "one", "would", "right",
                   "okay", 'could', 'know', 'make', 'something', 'yes', 'good',
                   "think", "oh", "maybe", 'kinda', 'actually', 'thing', 'things',
                   'gonna', 'wanna', 'really', 'hmm',
                   # The below set was after topic modeling
                   'cuz', 'huh', 'didn', 'wouldn', 'doesn', 'whatever', 'guys'
                  ]

complete_stop_words = stopwords.words('english') + words_to_remove

tf_vectorizer_grp1 = CountVectorizer(strip_accents = 'unicode',
                                     analyzer = 'word',
                                     stop_words = complete_stop_words,
                                     lowercase = True,
                                     token_pattern = r'\b[a-zA-Z]{3,}\b',
                                     max_df = 0.5, 
                                     min_df = 10)

tf_vectorizer_grp2 = CountVectorizer(strip_accents = 'unicode',
                                     analyzer = 'word',
                                     stop_words = complete_stop_words,
                                     lowercase = True,
                                     token_pattern = r'\b[a-zA-Z]{3,}\b',
                                     max_df = 0.5, 
                                     min_df = 10)
dtm_tf_grp1 = tf_vectorizer_grp1.fit_transform(group1_lines)
print("Document-Term Matrix size for Free Group:", dtm_tf_grp1.shape)
dtm_tf_grp2 = tf_vectorizer_grp2.fit_transform(group2_lines)
print("Document-Term Matrix size for D-C Group:", dtm_tf_grp2.shape)

Document-Term Matrix size for Free Group: (7641, 475)
Document-Term Matrix size for D-C Group: (3872, 285)


## 2.2 Compute the Term Frequency - Inverse Document Frequency
In order to use a scheme for weighting the document-term matrix, we use the popular **Inverse Document Frequency (IDF)** weighting measure.
The IDF is calculated as:

$IDF = log(N / n_j)$

Where $N$ is the total number of documents, and $n_j$ is the number of documents containing term $j$.
This system gives higher weightage to terms that occur in relatively fewer documents (and thus are more characteristic of the *topic* of that document)

The weighted matrix is called Term Frequency-Inverse Document Frequency, or **Tf-idf**, where each cell (term frequency) is multiplied by the weight (inverse document frequency)

In [4]:
tfidf_vectorizer_grp1 = TfidfVectorizer(**tf_vectorizer_grp1.get_params())
dtm_tfidf_grp1 = tfidf_vectorizer_grp1.fit_transform(group1_lines)

tfidf_vectorizer_grp2 = TfidfVectorizer(**tf_vectorizer_grp2.get_params())
dtm_tfidf_grp2 = tfidf_vectorizer_grp2.fit_transform(group2_lines)

## 2.3 Compute Latent Dirichlet Allocation (LDA)
The following explanation is from [Edwin Chen's website](http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/)

LDA represents documents as mixtures of topics that spit out words with certain probabilities. It assumes that documents are produced in the following fashion: when writing each document, you:

* Decide on the number of words N the document will have (say, according to a Poisson distribution).
* Choose a topic mixture for the document (according to a Dirichlet distribution over a fixed set of K topics). For example, assuming that we have the two food and cute animal topics above, you might choose the document to consist of 1/3 food and 2/3 cute animals.
* Generate each word w_i in the document by:
    * First picking a topic (according to the multinomial distribution that you sampled above; for example, you might pick the food topic with 1/3 probability and the cute animals topic with 2/3 probability).
    * Using the topic to generate the word itself (according to the topic’s multinomial distribution). For example, if we selected the food topic, we might generate the word “broccoli” with 30% probability, “bananas” with 15% probability, and so on.
* Assuming this generative model for a collection of documents, LDA then tries to backtrack from the documents to find a set of topics that are likely to have generated the collection.

### Example

Let’s make an example. According to the above process, when generating some particular document D, you might

* Pick 5 to be the number of words in D.
* Decide that D will be 1/2 about food and 1/2 about cute animals. (Number of topics = 2)
* Pick the first word to come from the food topic, which then gives you the word “broccoli”.
* Pick the second word to come from the cute animals topic, which gives you “panda”.
* Pick the third word to come from the cute animals topic, giving you “adorable”.
* Pick the fourth word to come from the food topic, giving you “cherries”.
* Pick the fifth word to come from the food topic, giving you “eating”.
So the document generated under the LDA model will be “broccoli panda adorable cherries eating” (note that LDA is a bag-of-words model).


In [5]:
# for TF DTM
lda_tf_grp1 = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tf_grp1.fit(dtm_tf_grp1)
lda_tf_grp2 = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tf_grp2.fit(dtm_tf_grp2)
# for TFIDF DTM
lda_tfidf_grp1 = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tfidf_grp1.fit(dtm_tfidf_grp1)
lda_tfidf_grp2 = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tfidf_grp2.fit(dtm_tfidf_grp2)

C:\Anaconda3\lib\site-packages\IPython\core\formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
C:\Anaconda3\lib\site-packages\IPython\core\formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
C:\Anaconda3\lib\site-packages\IPython\core\formatters.py:672: DeprecationWarning: PlainTextFormatter._type_printers_default is deprecated: use @default decorator instead.
  def _type_printers_default(self):
C:\Anaconda3\lib\site-packages\IPython\core\formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=20, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

## 2.4 Visualize Free Brainstorming Topics Using Term Frequencies (unweighted)
Using PyLDAvis, visualize topics and salient terms. To focus on terms that are more salient to each topic, set $\lambda$ to 0.4.

In [6]:
# pyLDAvis.sklearn.prepare(lda_tf_grp1, dtm_tf_grp1, tf_vectorizer_grp1)

In [7]:
# pyLDAvis.sklearn.prepare(lda_tf_grp2, dtm_tf_grp2, tf_vectorizer_grp2)

## 2.4 Visualize Free Brainstorming Topics Using TF-IDF (weighted)
Using PyLDAvis, visualize topics and salient terms. To focus on terms that are more salient to each topic, set $\lambda$ to 0.4.

In [8]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp1, dtm_tfidf_grp1, tfidf_vectorizer_grp1)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
1      5.994468        1       1 -0.060796 -0.214100
12     5.989569        1       2 -0.105752  0.254025
17     5.903551        1       3  0.061217  0.036852
5      5.848735        1       4  0.009524  0.058534
13     5.697429        1       5 -0.028415 -0.005506
16     5.637445        1       6  0.079156 -0.054589
6      5.438905        1       7 -0.251964  0.049524
11     5.303119        1       8 -0.205761 -0.144295
3      5.230371        1       9 -0.039660  0.045064
0      5.041724        1      10  0.074436 -0.028434
15     4.954635        1      11  0.150481 -0.072671
7      4.845897        1      12 -0.160479  0.064398
8      4.767281        1      13 -0.076643 -0.143236
10     4.700271        1      14  0.129108 -0.023178
14     4.695089        1      15  0.067469  0.220391
19     4.395403        1      16  0.110860  0.069112
9      4.311603        1      17  0.057544 -0.035217
18     4.289307        1      18  0.087250 -0.038927
4      3.514421        1      19  0.052357 -0.019162
2      3.440776        1      20  0.050066 -0.018586, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
457   Default  103.000000        well  103.000000  30.0000  30.0000
241   Default  101.000000        mean  101.000000  29.0000  29.0000
194   Default  102.000000        idea  102.000000  28.0000  28.0000
315   Default   90.000000         put   90.000000  27.0000  27.0000
420   Default   72.000000         toy   72.000000  26.0000  26.0000
169   Default   70.000000         got   70.000000  25.0000  25.0000
434   Default   65.000000         two   65.000000  24.0000  24.0000
396   Default   56.000000        sure   56.000000  23.0000  23.0000
221   Default   73.000000      little   73.000000  22.0000  22.0000
84    Default   63.000000        cool   63.000000  21.0000  21.0000
159   Default   86.000000         get   86.000000  20.0000  20.0000
305   Default   60.000000      pretty   60.000000  19.0000  19.0000
453   Default   58.000000         way   58.000000  18.0000  18.0000
452   Default   56.000000       water   56.000000  17.0000  17.0000
340   Default   56.000000         see   56.000000  16.0000  16.0000
104   Default   51.000000        draw   51.000000  15.0000  15.0000
261   Default   53.000000        need   53.000000  14.0000  14.0000
308   Default   48.000000    probably   48.000000  13.0000  13.0000
216   Default   62.000000         let   62.000000  12.0000  12.0000
205   Default   49.000000        kind   49.000000  11.0000  11.0000
24    Default   56.000000        back   56.000000  10.0000  10.0000
403   Default   45.000000        tank   45.000000   9.0000   9.0000
63    Default   43.000000         car   43.000000   8.0000   8.0000
448   Default   47.000000        want   47.000000   7.0000   7.0000
410   Default   43.000000      though   43.000000   6.0000   6.0000
26    Default   51.000000        ball   51.000000   5.0000   5.0000
16    Default   40.000000    anything   40.000000   4.0000   4.0000
234   Default   39.000000         man   39.000000   3.0000   3.0000
252   Default   41.000000      motion   41.000000   2.0000   2.0000
181   Default   37.000000        hard   37.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
419   Topic20    5.971583     towards    6.876223   3.2284  -3.8390
139   Topic20    5.961378        fire    6.866019   3.2282  -3.8407
314   Topic20    0.061666     pushing    4.174539  -0.8455  -8.4120
38    Topic20    0.061666     bicycle    4.393410  -0.8966  -8.4120
289   Topic20    0.061666       piece    5.011639  -1.0283  -8.4120
359   Topic20    0.061666         sit    5.222105  -1.0694  -8.4120
125   Topic20    0.061666     explain    5.252855  -1.0753  -8.4120
447   Topic20    0.061666       walls    5.296933  -1.0837  -8.4120
52 

## 2.4 Visualize D-C Brainstorming Topics Using TF-IDF (weighted)

In [9]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp2, dtm_tfidf_grp2, tfidf_vectorizer_grp2)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
4      7.064412        1       1  0.158550 -0.209916
6      6.475361        1       2 -0.245184 -0.109384
12     6.307357        1       3  0.193013 -0.184658
18     6.117692        1       4  0.033337  0.050457
19     5.886003        1       5  0.073540 -0.043244
8      5.600260        1       6 -0.181425 -0.088934
14     5.453948        1       7  0.031661  0.089044
13     5.034364        1       8  0.095150 -0.008130
2      4.747465        1       9 -0.033150  0.055392
0      4.719575        1      10  0.034412  0.185335
7      4.610244        1      11  0.102902 -0.031532
16     4.548703        1      12 -0.156026 -0.088265
5      4.499583        1      13 -0.018489  0.194987
9      4.422323        1      14  0.099205 -0.046039
3      4.416124        1      15  0.001866  0.084668
17     4.345037        1      16 -0.186004 -0.099517
15     4.149211        1      17 -0.007147  0.055338
11     4.102095        1      18  0.002635  0.069347
10     3.968668        1      19  0.000373  0.067560
1      3.531578        1      20  0.000780  0.057492, topic_info=     Category       Freq       Term      Total  loglift  logprob
term                                                            
60    Default  70.000000       cool  70.000000  30.0000  30.0000
130   Default  59.000000       idea  59.000000  29.0000  29.0000
201   Default  60.000000        put  60.000000  28.0000  28.0000
137   Default  61.000000       kind  61.000000  27.0000  27.0000
109   Default  53.000000        get  53.000000  26.0000  26.0000
271   Default  45.000000      water  45.000000  25.0000  25.0000
162   Default  51.000000       mean  51.000000  24.0000  24.0000
263   Default  40.000000        two  40.000000  23.0000  23.0000
43    Default  40.000000        car  40.000000  22.0000  22.0000
115   Default  51.000000        got  51.000000  21.0000  21.0000
270   Default  39.000000       want  39.000000  20.0000  20.0000
167   Default  31.000000     motion  31.000000  19.0000  19.0000
190   Default  29.000000       play  29.000000  18.0000  18.0000
70    Default  32.000000       draw  32.000000  17.0000  17.0000
106   Default  29.000000       game  29.000000  16.0000  16.0000
77    Default  28.000000       else  28.000000  15.0000  15.0000
145   Default  32.000000        let  32.000000  14.0000  14.0000
225   Default  27.000000       side  27.000000  13.0000  13.0000
5     Default  27.000000    alright  27.000000  12.0000  12.0000
149   Default  30.000000     little  30.000000  11.0000  11.0000
223   Default  26.000000     shoots  26.000000  10.0000  10.0000
3     Default  24.000000        air  24.000000   9.0000   9.0000
254   Default  40.000000        toy  40.000000   8.0000   8.0000
119   Default  27.000000      guess  27.000000   7.0000   7.0000
235   Default  28.000000      start  28.000000   6.0000   6.0000
131   Default  30.000000      ideas  30.000000   5.0000   5.0000
221   Default  23.000000      shoot  23.000000   4.0000   4.0000
173   Default  26.000000       need  26.000000   3.0000   3.0000
214   Default  26.000000        see  26.000000   2.0000   2.0000
176   Default  21.000000       nice  21.000000   1.0000   1.0000
...       ...        ...        ...        ...      ...      ...
143   Topic20   0.057807  launching   5.234296  -1.1625  -7.7790
238   Topic20   0.057807      stuck   5.461964  -1.2050  -7.7790
163   Topic20   0.057812     middle   5.526150  -1.2166  -7.7789
86    Topic20   0.057807     except   5.626839  -1.2348  -7.7790
180   Topic20   0.057812      paper   5.665513  -1.2415  -7.7789
79    Topic20   0.057807     enough   5.701402  -1.2479  -7.7790
161   Topic20   0.057807       many   5.723865  -1.2519  -7.7790
232   Topic20   0.057808       spin   5.773392  -1.2605  -7.7790
165   Topic20   0.057808       mine   5.818075  -1.2682  -7.7790
20    Topic20   0.057807       bear   5.921196  -1.2858  -7.7

In [10]:
filesListFreeDiv = openFiles(fileNamesFreeDivergent, textPathFree)
filesListFreeConv = openFiles(fileNamesFreeConvergent, textPathFree)
filesListDCDiv = openFiles(fileNamesDCDivergent, textPathDC)
filesListDCConv = openFiles(fileNamesDCConvergent, textPathDC)

group1_raw_div = removeSpecials(filesListFreeDiv)
group1_raw_conv = removeSpecials(filesListFreeConv)
group2_raw_div = removeSpecials(filesListDCDiv)
group2_raw_conv = removeSpecials(filesListDCConv)

group1Div = removeObservations(group1_raw_div)
group1Conv = removeObservations(group1_raw_conv)
group2Div = removeObservations(group2_raw_div)
group2Conv = removeObservations(group2_raw_conv)

group1_div_lines = group1Div.split('\n')
group1_conv_lines = group1Conv.split('\n')
group2_div_lines = group2Div.split('\n')
group2_conv_lines = group2Conv.split('\n')

print("Number of lines in the Free Group (divergent process):", len(group1_div_lines))
print("Number of lines in the Free Group (convergent process):", len(group1_conv_lines))
print("Number of lines in the D-C Group (divergent process):", len(group2_div_lines))
print("Number of lines in the D-C Group (convergent process):", len(group2_conv_lines))
print("Each line will be considered a separate document for computing the Document-Term Matrix")

Number of lines in the Free Group (divergent process): 5023
Number of lines in the Free Group (convergent process): 2619
Number of lines in the D-C Group (divergent process): 3872
Number of lines in the D-C Group (convergent process): 2769
Each line will be considered a separate document for computing the Document-Term Matrix


In [11]:
tf_vectorizer_grp1_div = CountVectorizer(strip_accents = 'unicode',
                                         analyzer = 'word',
                                         stop_words = complete_stop_words,
                                         lowercase = True,
                                         token_pattern = r'\b[a-zA-Z]{3,}\b',
                                         max_df = 0.5, 
                                         min_df = 10)

tf_vectorizer_grp1_conv = CountVectorizer(strip_accents = 'unicode',
                                          analyzer = 'word',
                                          stop_words = complete_stop_words,
                                          lowercase = True,
                                          token_pattern = r'\b[a-zA-Z]{3,}\b',
                                          max_df = 0.5, 
                                          min_df = 10)

tf_vectorizer_grp2_div = CountVectorizer(strip_accents = 'unicode',
                                         analyzer = 'word',
                                         stop_words = complete_stop_words,
                                         lowercase = True,
                                         token_pattern = r'\b[a-zA-Z]{3,}\b',
                                         max_df = 0.5, 
                                         min_df = 10)

tf_vectorizer_grp2_conv = CountVectorizer(strip_accents = 'unicode',
                                          analyzer = 'word',
                                          stop_words = complete_stop_words,
                                          lowercase = True,
                                          token_pattern = r'\b[a-zA-Z]{3,}\b',
                                          max_df = 0.5, 
                                          min_df = 10)
dtm_tf_grp1_div = tf_vectorizer_grp1.fit_transform(group1_div_lines)
print("Document-Term Matrix size for Free Group (divergent process):", dtm_tf_grp1_div.shape)
dtm_tf_grp1_conv = tf_vectorizer_grp1.fit_transform(group1_conv_lines)
print("Document-Term Matrix size for Free Group (convergent process):", dtm_tf_grp1_conv.shape)

dtm_tf_grp2_div = tf_vectorizer_grp2.fit_transform(group2_div_lines)
print("Document-Term Matrix size for Free Group (divergent process):", dtm_tf_grp2_div.shape)
dtm_tf_grp2_conv = tf_vectorizer_grp2.fit_transform(group2_conv_lines)
print("Document-Term Matrix size for Free Group (convergent process):", dtm_tf_grp2_conv.shape)

Document-Term Matrix size for Free Group (divergent process): (5023, 353)
Document-Term Matrix size for Free Group (convergent process): (2619, 159)
Document-Term Matrix size for Free Group (divergent process): (3872, 285)
Document-Term Matrix size for Free Group (convergent process): (2769, 184)


In [17]:
tfidf_vectorizer_grp1_div = TfidfVectorizer(**tf_vectorizer_grp1_div.get_params())
dtm_tfidf_grp1_div = tfidf_vectorizer_grp1_div.fit_transform(group1_div_lines)
tfidf_vectorizer_grp1_conv = TfidfVectorizer(**tf_vectorizer_grp1_conv.get_params())
dtm_tfidf_grp1_conv = tfidf_vectorizer_grp1_conv.fit_transform(group1_conv_lines)

tfidf_vectorizer_grp2_div = TfidfVectorizer(**tf_vectorizer_grp2_div.get_params())
dtm_tfidf_grp2_div = tfidf_vectorizer_grp2_div.fit_transform(group2_div_lines)
tfidf_vectorizer_grp2_conv = TfidfVectorizer(**tf_vectorizer_grp2_conv.get_params())
dtm_tfidf_grp2_conv = tfidf_vectorizer_grp2_conv.fit_transform(group2_conv_lines)

# for TFIDF DTM
lda_tfidf_grp1_div = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tfidf_grp1_div.fit(dtm_tfidf_grp1_div)
lda_tfidf_grp1_conv = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tfidf_grp1_conv.fit(dtm_tfidf_grp1_conv)

lda_tfidf_grp2_div = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tfidf_grp2_div.fit(dtm_tfidf_grp2_div)
lda_tfidf_grp2_conv = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tfidf_grp2_conv.fit(dtm_tfidf_grp2_conv)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=20, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

### Topics in The Divergent Process of the Free Brainstorming Group

In [18]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp1_div, dtm_tfidf_grp1_div, tfidf_vectorizer_grp1_div)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
15     7.466143        1       1  0.217319 -0.104609
9      6.417984        1       2  0.076876  0.220203
10     6.047491        1       3 -0.131174  0.062693
1      6.015128        1       4  0.169946 -0.213626
18     5.905615        1       5  0.090036  0.164713
17     5.464090        1       6  0.131794 -0.174539
5      5.005672        1       7 -0.041165 -0.034589
4      4.917853        1       8 -0.242433 -0.090904
16     4.842113        1       9  0.008312 -0.032784
12     4.807820        1      10 -0.208669 -0.101685
7      4.802515        1      11  0.011237  0.075734
13     4.792428        1      12  0.042455  0.117703
0      4.747667        1      13  0.041957 -0.038774
19     4.653966        1      14 -0.035293 -0.005489
2      4.403412        1      15  0.069070  0.147005
8      4.250038        1      16 -0.037984 -0.000672
14     4.215601        1      17 -0.046830 -0.014631
11     4.205943        1      18 -0.032507  0.001101
6      3.610102        1      19 -0.053921  0.018993
3      3.428418        1      20 -0.029025  0.004156, topic_info=     Category       Freq        Term      Total  loglift  logprob
term                                                             
144   Default  82.000000        idea  82.000000  30.0000  30.0000
239   Default  65.000000         put  65.000000  29.0000  29.0000
337   Default  60.000000       water  60.000000  28.0000  28.0000
312   Default  57.000000         toy  57.000000  27.0000  27.0000
124   Default  52.000000         got  52.000000  26.0000  26.0000
184   Default  62.000000        mean  62.000000  25.0000  25.0000
63    Default  49.000000        cool  49.000000  24.0000  24.0000
166   Default  55.000000      little  55.000000  23.0000  23.0000
340   Default  55.000000        well  55.000000  22.0000  22.0000
338   Default  44.000000         way  44.000000  21.0000  21.0000
19    Default  41.000000        ball  41.000000  20.0000  20.0000
232   Default  44.000000      pretty  44.000000  19.0000  19.0000
48    Default  40.000000         car  40.000000  18.0000  18.0000
162   Default  50.000000         let  50.000000  17.0000  17.0000
259   Default  38.000000         see  38.000000  16.0000  16.0000
117   Default  51.000000         get  51.000000  15.0000  15.0000
121   Default  31.000000        goes  31.000000  14.0000  14.0000
334   Default  31.000000        want  31.000000  13.0000  13.0000
61    Default  31.000000     control  31.000000  12.0000  12.0000
200   Default  31.000000        need  31.000000  11.0000  11.0000
17    Default  40.000000        back  40.000000  10.0000  10.0000
313   Default  34.000000        toys  34.000000   9.0000   9.0000
146   Default  27.000000      inside  27.000000   8.0000   8.0000
46    Default  30.000000       candy  30.000000   7.0000   7.0000
169   Default  27.000000       looks  27.000000   6.0000   6.0000
311   Default  29.000000         top  29.000000   5.0000   5.0000
128   Default  30.000000         gun  30.000000   4.0000   4.0000
265   Default  29.000000      shoots  29.000000   3.0000   3.0000
308   Default  28.000000        time  28.000000   2.0000   2.0000
295   Default  31.000000       stuff  31.000000   1.0000   1.0000
...       ...        ...         ...        ...      ...      ...
161   Topic20   0.059419        less   4.785898  -1.0157  -7.9906
56    Topic20   0.059418     combine   4.804020  -1.0195  -7.9906
288   Topic20   0.059418     started   4.811649  -1.0211  -7.9906
110   Topic20   0.059418       forth   4.916498  -1.0427  -7.9906
38    Topic20   0.059418       bring   5.038045  -1.0671  -7.9906
22    Topic20   0.059418        base   5.056581  -1.0708  -7.9906
77    Topic20   0.059418  electronic   5.081019  -1.0756  -7.9906
300   Topic20   0.059418      target   5.115698  -1.0824  -7.9906
194   Topic20   0.059418       mouth   5.244833  -1.1073  -7.9906
335   Topic20   0.0

### Topics in The Convergent Process of the Free Brainstorming Group

In [19]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp1_conv, dtm_tfidf_grp1_conv, tfidf_vectorizer_grp1_conv)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
15     6.936294        1       1 -0.161469 -0.133616
5      5.542627        1       2  0.097348 -0.213032
19     5.524840        1       3  0.002342  0.046259
14     5.364144        1       4  0.273475  0.061011
3      5.337601        1       5  0.082195  0.002875
1      5.290963        1       6 -0.127751  0.116067
17     5.228028        1       7 -0.040090  0.035424
10     5.175332        1       8  0.002156  0.066822
16     5.164374        1       9 -0.078818 -0.039546
13     5.061796        1      10  0.237114  0.102385
6      5.041845        1      11 -0.048510 -0.238553
8      4.952815        1      12 -0.125708  0.154611
11     4.776755        1      13  0.000595  0.031935
18     4.766432        1      14  0.019188 -0.174786
2      4.685473        1      15  0.000779  0.031845
0      4.485285        1      16  0.000538  0.056474
7      4.344462        1      17  0.040825  0.009575
4      4.285121        1      18 -0.159044  0.121416
12     4.266350        1      19  0.000137  0.021894
9      3.769462        1      20 -0.015304 -0.059060, topic_info=     Category       Freq         Term      Total  loglift  logprob
term                                                              
62    Default  43.000000         idea  43.000000  30.0000  30.0000
51    Default  46.000000          get  46.000000  29.0000  29.0000
149   Default  44.000000         well  44.000000  28.0000  28.0000
79    Default  44.000000         mean  44.000000  27.0000  27.0000
127   Default  38.000000         tank  38.000000  26.0000  26.0000
100   Default  32.000000     probably  32.000000  25.0000  25.0000
54    Default  31.000000          got  31.000000  24.0000  24.0000
102   Default  30.000000          put  30.000000  23.0000  23.0000
77    Default  24.000000          man  24.000000  22.0000  22.0000
99    Default  26.000000       pretty  26.000000  21.0000  21.0000
142   Default  33.000000          two  33.000000  20.0000  20.0000
13    Default  23.000000         ball  23.000000  19.0000  19.0000
146   Default  24.000000         want  24.000000  18.0000  18.0000
45    Default  24.000000        final  24.000000  17.0000  17.0000
11    Default  24.000000         back  24.000000  16.0000  16.0000
87    Default  26.000000         need  26.000000  15.0000  15.0000
69    Default  22.000000          let  22.000000  14.0000  14.0000
130   Default  21.000000       though  21.000000  13.0000  13.0000
148   Default  22.000000          way  22.000000  12.0000  12.0000
119   Default  20.000000       spring  20.000000  11.0000  11.0000
136   Default  24.000000          toy  24.000000  10.0000  10.0000
74    Default  21.000000        looks  21.000000   9.0000   9.0000
153   Default  21.000000         wind  21.000000   8.0000   8.0000
71    Default  25.000000       little  25.000000   7.0000   7.0000
36    Default  18.000000      drawing  18.000000   6.0000   6.0000
30    Default  20.000000         cool  20.000000   5.0000   5.0000
35    Default  23.000000         draw  23.000000   4.0000   4.0000
25    Default  17.000000         come  17.000000   3.0000   3.0000
132   Default  18.000000        three  18.000000   2.0000   2.0000
107   Default  20.000000          see  20.000000   1.0000   1.0000
...       ...        ...          ...        ...      ...      ...
1     Topic20   0.057830        alarm   6.620581  -1.4622  -7.2540
14    Topic20   0.057710    basically   6.672937  -1.4721  -7.2561
101   Topic20   0.057716      profile   6.797398  -1.4905  -7.2560
20    Topic20   0.057709       bounce   6.799353  -1.4909  -7.2561
9     Topic20   0.057706       around   6.909459  -1.5071  -7.2562
47    Topic20   0.057711         five   6.925023  -1.5092  -7.2561
48    Topic20   0.057713      forward   6.952556  -1.5131  -7.2561
116   Topic20   0.057706      someone   7.060684  -1.5287  -7.2562
7     Topic20   0.057709          arm   7.19

### Topics in The Divergent Process of the D-C Brainstorming Group

In [20]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp2_div, dtm_tfidf_grp2_div, tfidf_vectorizer_grp2_div)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
4      7.064412        1       1  0.158550 -0.209916
6      6.475361        1       2 -0.245184 -0.109384
12     6.307357        1       3  0.193013 -0.184658
18     6.117692        1       4  0.033337  0.050457
19     5.886003        1       5  0.073540 -0.043244
8      5.600260        1       6 -0.181425 -0.088934
14     5.453948        1       7  0.031661  0.089044
13     5.034364        1       8  0.095150 -0.008130
2      4.747465        1       9 -0.033150  0.055392
0      4.719575        1      10  0.034412  0.185335
7      4.610244        1      11  0.102902 -0.031532
16     4.548703        1      12 -0.156026 -0.088265
5      4.499583        1      13 -0.018489  0.194987
9      4.422323        1      14  0.099205 -0.046039
3      4.416124        1      15  0.001866  0.084668
17     4.345037        1      16 -0.186004 -0.099517
15     4.149211        1      17 -0.007147  0.055338
11     4.102095        1      18  0.002635  0.069347
10     3.968668        1      19  0.000373  0.067560
1      3.531578        1      20  0.000780  0.057492, topic_info=     Category       Freq       Term      Total  loglift  logprob
term                                                            
60    Default  70.000000       cool  70.000000  30.0000  30.0000
130   Default  59.000000       idea  59.000000  29.0000  29.0000
201   Default  60.000000        put  60.000000  28.0000  28.0000
137   Default  61.000000       kind  61.000000  27.0000  27.0000
109   Default  53.000000        get  53.000000  26.0000  26.0000
271   Default  45.000000      water  45.000000  25.0000  25.0000
162   Default  51.000000       mean  51.000000  24.0000  24.0000
263   Default  40.000000        two  40.000000  23.0000  23.0000
43    Default  40.000000        car  40.000000  22.0000  22.0000
115   Default  51.000000        got  51.000000  21.0000  21.0000
270   Default  39.000000       want  39.000000  20.0000  20.0000
167   Default  31.000000     motion  31.000000  19.0000  19.0000
190   Default  29.000000       play  29.000000  18.0000  18.0000
70    Default  32.000000       draw  32.000000  17.0000  17.0000
106   Default  29.000000       game  29.000000  16.0000  16.0000
77    Default  28.000000       else  28.000000  15.0000  15.0000
145   Default  32.000000        let  32.000000  14.0000  14.0000
225   Default  27.000000       side  27.000000  13.0000  13.0000
5     Default  27.000000    alright  27.000000  12.0000  12.0000
149   Default  30.000000     little  30.000000  11.0000  11.0000
223   Default  26.000000     shoots  26.000000  10.0000  10.0000
3     Default  24.000000        air  24.000000   9.0000   9.0000
254   Default  40.000000        toy  40.000000   8.0000   8.0000
119   Default  27.000000      guess  27.000000   7.0000   7.0000
235   Default  28.000000      start  28.000000   6.0000   6.0000
131   Default  30.000000      ideas  30.000000   5.0000   5.0000
221   Default  23.000000      shoot  23.000000   4.0000   4.0000
173   Default  26.000000       need  26.000000   3.0000   3.0000
214   Default  26.000000        see  26.000000   2.0000   2.0000
176   Default  21.000000       nice  21.000000   1.0000   1.0000
...       ...        ...        ...        ...      ...      ...
143   Topic20   0.057807  launching   5.234296  -1.1625  -7.7790
238   Topic20   0.057807      stuck   5.461964  -1.2050  -7.7790
163   Topic20   0.057812     middle   5.526150  -1.2166  -7.7789
86    Topic20   0.057807     except   5.626839  -1.2348  -7.7790
180   Topic20   0.057812      paper   5.665513  -1.2415  -7.7789
79    Topic20   0.057807     enough   5.701402  -1.2479  -7.7790
161   Topic20   0.057807       many   5.723865  -1.2519  -7.7790
232   Topic20   0.057808       spin   5.773392  -1.2605  -7.7790
165   Topic20   0.057808       mine   5.818075  -1.2682  -7.7790
20    Topic20   0.057807       bear   5.921196  -1.2858  -7.7

### Topics in The Convergent Process of the D-C Brainstorming Group

In [21]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp2_conv, dtm_tfidf_grp2_conv, tfidf_vectorizer_grp2_conv)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
19     6.839610        1       1 -0.246884 -0.085734
11     6.486500        1       2  0.024316  0.084326
0      6.050641        1       3 -0.262258 -0.098663
14     6.045769        1       4 -0.077340 -0.063030
7      5.945227        1       5  0.147030 -0.296117
13     5.867745        1       6  0.094869 -0.053745
8      5.810149        1       7  0.189109 -0.247383
17     5.630630        1       8  0.092826  0.092914
9      5.341901        1       9 -0.214485 -0.086752
15     4.855224        1      10  0.044566  0.083888
10     4.698242        1      11  0.035180  0.071841
1      4.499051        1      12 -0.022156  0.098790
12     4.462751        1      13 -0.006013  0.070348
4      4.363526        1      14 -0.003064  0.069461
6      4.170160        1      15 -0.004270  0.061025
2      4.093095        1      16  0.019279  0.093621
16     4.019360        1      17  0.027951  0.076902
18     3.779230        1      18  0.076993  0.064656
3      3.724357        1      19  0.054812  0.008992
5      3.316831        1      20  0.029536  0.054661, topic_info=     Category       Freq       Term      Total  loglift  logprob
term                                                            
73    Default  44.000000        got  44.000000  30.0000  30.0000
105   Default  40.000000       mean  40.000000  29.0000  29.0000
92    Default  37.000000        let  37.000000  28.0000  28.0000
81    Default  38.000000       idea  38.000000  27.0000  27.0000
14    Default  40.000000       ball  40.000000  26.0000  26.0000
130   Default  41.000000        put  41.000000  25.0000  25.0000
90    Default  37.000000       kind  37.000000  24.0000  24.0000
181   Default  30.000000     wheels  30.000000  23.0000  23.0000
26    Default  44.000000        car  44.000000  22.0000  22.0000
35    Default  32.000000       cool  32.000000  21.0000  21.0000
176   Default  33.000000       want  33.000000  20.0000  20.0000
137   Default  23.000000        say  23.000000  19.0000  19.0000
68    Default  33.000000        get  33.000000  18.0000  18.0000
172   Default  39.000000        two  39.000000  17.0000  17.0000
96    Default  21.000000       look  21.000000  16.0000  16.0000
160   Default  24.000000   thinking  24.000000  15.0000  15.0000
178   Default  25.000000        way  25.000000  14.0000  14.0000
38    Default  25.000000  different  25.000000  13.0000  13.0000
57    Default  28.000000      final  28.000000  12.0000  12.0000
13    Default  22.000000       back  22.000000  11.0000  11.0000
40    Default  24.000000       draw  24.000000  10.0000  10.0000
94    Default  22.000000     little  22.000000   9.0000   9.0000
159   Default  23.000000       take  23.000000   8.0000   8.0000
153   Default  21.000000      start  21.000000   7.0000   7.0000
122   Default  22.000000      point  22.000000   6.0000   6.0000
87    Default  18.000000      jumps  18.000000   5.0000   5.0000
179   Default  26.000000       well  26.000000   4.0000   4.0000
112   Default  34.000000       need  34.000000   3.0000   3.0000
161   Default  19.000000     though  19.000000   2.0000   2.0000
139   Default  22.000000        see  22.000000   1.0000   1.0000
...       ...        ...        ...        ...      ...      ...
58    Topic20   0.063317       find   7.073940  -1.3099  -7.2200
142   Topic20   0.063319        set   7.091465  -1.3123  -7.2200
21    Topic20   0.063317  boomerang   7.124920  -1.3170  -7.2200
22    Topic20   0.063315     bottom   7.303482  -1.3418  -7.2201
145   Topic20   0.063322     shoots   7.304894  -1.3419  -7.2199
69    Topic20   0.063315       gets   7.380328  -1.3523  -7.2201
169   Topic20   0.063319        try   7.486688  -1.3665  -7.2200
146   Topic20   0.063311       side   7.516864  -1.3707  -7.2201
156   Topic20   0.065438       suit  18.004847  -2.2111  -7.1871
168   Topic20   0.063984      trunk   9.968052  -1.6423  -7.2